In [2]:
import os
import pandas as pd
datas = []
for fold in os.listdir('folds'):
    for file in os.listdir(f'folds/{fold}'):
        data = pd.read_csv(f'folds/{fold}/{file}')
        datas.append(data)
combined_data = pd.concat(datas, ignore_index=True)
filtered_data = combined_data[['dlip_id', 'SMILES']]
filtered_data = filtered_data.drop_duplicates().reset_index(drop=True)
filtered_data.to_csv('mod_filtered_data.csv', index=False)

In [5]:
filtered_data.iloc[0,1]

'Cc1cccc(Cl)c1C(=O)N[C@@H](Cc1ccc(NC(=O)c2c(Cl)cccc2Cl)cc1)C(=O)O'

In [25]:
import requests
import json

def search_pubchem(smiles):
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/substructure/smiles/{smiles}/JSON"
    response = requests.get(url)
    if response.status_code == 200:
        return json.loads(response.text)
    else:
        return None

# Example SMILES for testing
test_smiles = "Cc1cccc(Cl)c1C(=O)N[C@@H](Cc1ccc(NC(=O)c2c(Cl)cccc2Cl)cc1)C(=O)O"

result = search_pubchem(test_smiles)

if result:
    print(json.dumps(result, indent=2))
else:
    print("No results found or an error occurred.")


No results found or an error occurred.


In [6]:
import pubchempy as pcp
import os
import tempfile
import pandas as pd
from rdkit import Chem
from tqdm import tqdm

# Create mol_pdb folder if it doesn't exist
mol_pdb_folder = "mol_pdb"
os.makedirs(mol_pdb_folder, exist_ok=True)
error_list = []

# Load filtered_data
filtered_data = pd.read_csv('filtered_data.csv')

for index, row in tqdm(filtered_data.iterrows()):
    smiles = row['SMILES']
    dlip_id = row['dlip_id']
    if os.path.exists(f'{mol_pdb_folder}/{dlip_id}.pdb'):
        continue
    try:
        compound = pcp.get_compounds(smiles, 'smiles', record_type='3d')
        if compound:
            cid = compound[0].cid
            
            # Download SDF to a temporary file
            with tempfile.NamedTemporaryFile(mode="w", suffix=".sdf", delete=False) as temp_sdf_file:
                pcp.download('SDF', temp_sdf_file.name, cid, 'cid', overwrite=True, record_type='3d')
                
                # Convert SDF to PDB
                mol_supplier = Chem.SDMolSupplier(temp_sdf_file.name)
                for mol in mol_supplier:
                    if mol is not None:
                        pdb_file_path = os.path.join(mol_pdb_folder, f"{dlip_id}.pdb")
                        Chem.MolToPDBFile(mol, pdb_file_path)
                        print(f"Processed {pdb_file_path}")
                        break
                else:
                    error_list.append((dlip_id,smiles))
                    print(f"No valid molecule found for {dlip_id}")

            # Clean up the temporary SDF file
            os.unlink(temp_sdf_file.name)
        else:
            error_list.append((dlip_id,smiles))
            print(f"No compound found for {dlip_id} and smiles: {smiles}")
    except Exception as e:
        error_list.append((dlip_id,smiles))
        print(f"Error processing {dlip_id}: {str(e)}")

print("All PDB files have been saved to the 'mol_pdb' folder.")
print(f"total length: {len(filtered_data)} and downloaded length: {len(os.listdir('mol_pdb'))}")

0it [00:00, ?it/s]

5it [00:00,  9.15it/s]

No compound found for T003RI and smiles: NCCCC[C@H](N)C(=O)NCC(=O)N[C@@H](Cc1ccc(O)cc1)C(=O)N[C@@H](Cc1ccc(O)cc1)C(=O)NCCCCCC(=O)N[C@@H](CCCCNC(=O)CCCC[C@H]1SC[C@H]2NC(=O)N[C@H]21)C(=O)NCCCCCC(=O)NCCCC[C@@H]1CNC(=O)C[C@H](CCCCNC(=O)CCCCCNC(=O)[C@H](CCCCNC(=O)CCCC[C@H]2SC[C@H]3NC(=O)N[C@H]32)NC(=O)CCCCCNC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)CNC(=O)[C@@H](N)CCCCN)NC[C@H](CCCCNC(=O)CCCCCNC(=O)[C@H](CCCCNC(=O)CCCC[C@H]2SC[C@H]3NC(=O)N[C@H]32)NC(=O)CCCCCNC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)CNC(=O)[C@@H](N)CCCCN)NC(=O)C1


7it [00:00,  6.90it/s]

No compound found for C005EP and smiles: CC(C)C[C@]1(CCO)C[C@H](CNC(=S)Nc2ccc(-c3c4ccc(=O)cc-4oc4cc(O)ccc34)c(C(=O)O)c2)ON1Cc1ccc(-c2ccccc2)cc1


...

Processed mol_pdb/P0003B.pdb


4318it [31:22,  1.59it/s]

No compound found for J100BO and smiles: CCNCc1cc(C)c(OCc2cccc(-c3cccc(COc4cc(OCc5cncc(C#N)c5)c(CNCC)cc4C)c3C)c2C)cc1OCc1cncc(C#N)c1


4319it [31:23,  1.55it/s]

Processed mol_pdb/P000FW.pdb
